# Create a Arbitrage Betting Strategy

#### Install PolyMarket API library

In [1]:
!pip install py-clob-client

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for eth-account>=0.13.0 from https://files.pythonhosted.org/packages/46/18/088fb250018cbe665bc2111974301b2d59f294a565aff7564c4df6878da2/eth_account-0.13.7-py3-none-any.whl.metadata
  Obtaining dependency information for eth-utils>=4.1.1 from https://files.pythonhosted.org/packages/c4/c6/0417a92e6a3fc9b85f5a8380d9f9d43b69ba836a90e45f79f9ae74d41e53/eth_utils-5.3.0-py3-none-any.whl.metadata
  Obtaining dependency information for poly_eip712_structs>=0.0.1 from https://files.pythonhosted.org/packages/5a/d7/ff1cfba1c3a3d5d6851d7bef5e4ad19710ed6d03e149dc183111d103acab/poly_eip712_structs-0.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for py-order-utils>=0.3.2 from https://files.pythonhosted.org/packages/29/68/b0a971b064b3236fce7307bd5c180409cccd9b207ec459274bdb4e401ec0/py_order_utils-0.3.2-py3-none-any.whl.metadata
  Obtaining dependency

### NYC Mayoral Race 2025

#### Event: NYC Mayoral Election:
https://polymarket.com/event/new-york-city-mayoral-election?tid=1755656631578

#### Markets: 
- Mamdani
- Cuomo
- Adams
- Silwa

#### Get single event data using gamma api

In [24]:
import requests

r = requests.get("https://gamma-api.polymarket.com/events?id=23246")
nyc_mayoral_election_event = r.json()
nyc_mayoral_election_event

[{'id': '23246',
  'ticker': 'new-york-city-mayoral-election',
  'slug': 'new-york-city-mayoral-election',
  'title': 'New York City Mayoral Election',
  'description': 'The 2025 New York City mayoral election will be held on November 4, 2025.\n\nThis market will resolve according to the candidate wins the election.\n\nThe primary resolution source for this market will be a consensus of credible reporting, however if there is any ambiguity in the results this market will resolve according to official information from New York City.\n\n\n',
  'resolutionSource': '',
  'startDate': '2025-04-22T16:14:22.792059Z',
  'creationDate': '2025-04-22T16:14:22.792054Z',
  'endDate': '2025-11-04T12:00:00Z',
  'image': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/new-york-city-mayoral-election-al8KR4km2AQB.png',
  'icon': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/new-york-city-mayoral-election-al8KR4km2AQB.png',
  'active': True,
  'closed': False,
  'archived': False,
  'new': 

In [26]:
for market in nyc_mayoral_election_event[0]['markets']:
    print(market['id'], market['question'], 'outcomePrices' in market and market['outcomePrices'])
    print('clobTokenIds' in market and market['clobTokenIds'])
    print("----------------------------")

538928 Will Eric Adams win the 2025 NYC mayoral election? ["0.052", "0.948"]
["12541509255877010177934715422358422748052378132651567836811989084820478865881", "557149893577824223387740663828120572195409127574902734254512226734855221907"]
----------------------------
538929 Will Andrew Cuomo win the 2025 NYC mayoral election? ["0.101", "0.899"]
["72685162394098505217895638060393901041260225434938300730127268362092284806692", "104468181147316868388088006861839293041095272602974154655578369735976654024471"]
----------------------------
538930 Will Curtis Sliwa win the 2025 NYC mayoral election? ["0.0065", "0.9935"]
["106645483208866512069057468273735379467092160028743318106715987586591689242591", "42541673615301895829890290486226257940966769125829226067368474110048691276042"]
----------------------------
538931 Will Jim Walden win the 2025 NYC mayoral election? ["0.0045", "0.9955"]
["10142872462975275995759595767341515402450823584364917228661247714894341308969", "3323369066986882543667397

#### Get websocket data using Order Book API

In [ ]:
import json
import threading
from websocket import WebSocketApp

# WebSocket endpoint for Polymarket CLOB service
WS_URL_BASE = "wss://ws-subscriptions-clob.polymarket.com"

# Your target tokens (clobTokenIds)
ASSET_IDS = [
    "33945469250963963541781051637999677727672635213493648594066577298999471399137",  # Mamdani YES
    "105832362350788616148612362642992403996714020918558917275151746177525518770551",   # Mamdani NO
    "72685162394098505217895638060393901041260225434938300730127268362092284806692",  # Cuomo YES
    "104468181147316868388088006861839293041095272602974154655578369735976654024471"   # Cuomo NO
]

CHANNEL_TYPE = "market"  # use market for public price/book updates

class PolymarketWebSocket:
    def __init__(self, url_base, channel_type, asset_ids):
        self.url = f"{url_base}/ws/{channel_type}"
        self.channel_type = channel_type
        self.asset_ids = asset_ids
        self.ws = WebSocketApp(
            self.url,
            on_open=self.on_open,
            on_message=self.on_message,
            on_error=self.on_error,
            on_close=self.on_close,
        )

    def on_open(self, ws):
        print(f"Connected to {self.url}")
        # Subscribe to assets
        subscribe_payload = {
            "assets_ids": self.asset_ids,
            "type": self.channel_type
        }
        ws.send(json.dumps(subscribe_payload))
        # Start periodic ping to keep connection alive
        threading.Thread(target=self.ping, args=(ws,), daemon=True).start()

    def on_message(self, ws, message):
        msg = json.loads(message)
        event_type = msg.get("event_type")
        print(f"\nReceived message—event_type: {event_type}")
        print(json.dumps(msg, indent=2))

        # Example: handle different types
        if event_type == "book":
            # full order book snapshot/update
            self.handle_book(msg)
        elif event_type == "price_change":
            self.handle_price_change(msg)
        elif event_type == "last_trade_price":
            self.handle_last_trade(msg)
        elif event_type == "tick_size_change":
            self.handle_tick_size_change(msg)

    def on_error(self, ws, error):
        print("WebSocket error:", error)

    def on_close(self, ws, close_status_code, close_msg):
        print("WebSocket closed:", close_status_code, close_msg)

    def ping(self, ws):
        import time
        while True:
            ws.send("PING")
            time.sleep(10)

    def run(self):
        self.ws.run_forever()

    # Handler stubs — customize as needed
    def handle_book(self, msg):
        # Example: show top of bids/asks
        bids = msg.get("bids") or msg.get("buys")  # depending on naming
        asks = msg.get("asks") or msg.get("sells")
        print("Top bids:", bids[:3] if bids else bids)
        print("Top asks:", asks[:3] if asks else asks)

    def handle_price_change(self, msg):
        print("Price change update:", msg.get("changes"))

    def handle_last_trade(self, msg):
        print("Last trade price:", msg.get("price"), "size:", msg.get("size"))

    def handle_tick_size_change(self, msg):
        print("Tick size change:", msg.get("old_tick_size"), "→", msg.get("new_tick_size"))

if __name__ == "__main__":
    client = PolymarketWebSocket(WS_URL_BASE, CHANNEL_TYPE, ASSET_IDS)
    client.run()


Connected to wss://ws-subscriptions-clob.polymarket.com/ws/market


Exception in thread Thread-8 (ping):
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\user\anaconda3\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\user\AppData\Local\Temp\ipykernel_27220\778558047.py", line 66, in ping
  File "C:\Users\user\anaconda3\Lib\site-packages\websocket\_app.py", line 181, in send
    raise WebSocketConnectionClosedException(
websocket._exceptions.WebSocketConnectionClosedException: Connection is already closed.
